# Planning

This notebook demonstrates planning using CoT.

In [1]:
#%pip install -qU langchain-google-vertexai

In [1]:
from dotenv import load_dotenv

load_dotenv("../genai_agents/keys.env");

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain_core.output_parsers import StrOutputParser

## 1. LLMs are improving ...

for many questions, they will do a chain-of-thought and come up with the right answer.

In [3]:
model = ChatVertexAI(model="gemini-1.5-flash")

def ask_question(question, chain_of_thought=False):
    if chain_of_thought:
        preamble = """
        Write down the steps to solve this question, and work through the steps
        one-by-one to come up with the answer
        """
    else:
        preamble = ""
   
    prompt_template = PromptTemplate.from_template(
        preamble + "{question}"
    )
    parser = StrOutputParser()
    chain = prompt_template | model | parser

    answer = chain.invoke({
        "question": question,
    })
    print(answer)

In [6]:
ask_question("Who is the Mayor of the capital of the third most populated state in the United States?")

Here's how to solve this:

1. **Third most populated state:**  The third most populated state in the US is Florida.
2. **Capital of Florida:** The capital of Florida is Tallahassee.
3. **Mayor of Tallahassee:** The current Mayor of Tallahassee is  **John Dailey**. 

Therefore, the Mayor of the capital of the third most populated state in the United States is **John Dailey**. 



## List and Count

In [8]:
ask_question("How many states in the US have capital cities that end in City?",
            chain_of_thought=False)

There are **zero** states in the US that have capital cities that end in "City". 

While many state capitals have "City" in their names (e.g., Oklahoma City,  Phoenix City, etc.),  none of them end with "City". 



In [7]:
ask_question("How many states in the US have capital cities that end in City?",
            chain_of_thought=True)

Here's how we can solve this:

**Steps:**

1. **List US states and their capitals:** We need to know the capital cities of all US states. 
2. **Identify capitals ending in "City":**  We'll go through our list and check if each capital ends in "City".
3. **Count the matching capitals:** We'll keep track of how many capitals we find that end in "City".

**Let's do it:**

1. **List of US states and capitals:**  (You can use a reference like Wikipedia or a list of US states and capitals)

2. **Identify capitals ending in "City":**

   * **Alabama:** Montgomery (No)
   * **Alaska:** Juneau (No)
   * **Arizona:** Phoenix (No) 
   * **Arkansas:** Little Rock (No) 
   * ...  (Continue going through all the states)

3. **Count the matching capitals:** As we go through the list, we find:

   * **Kansas:** Topeka (No)
   * **Missouri:** Jefferson City (Yes!)
   * **New York:** Albany (No)
   * **Oklahoma:** Oklahoma City (Yes!) 

   ... and so on.

**Answer:** After going through all the states, 

In [7]:
## List and Count using Dynamic Planning
def create_plan(question): 
    plan_template = PromptTemplate.from_template(
       """What are the steps to solve the following question?
        Do not provide the answer, only the steps. Return the result
        as a list
        {question}
        """
    )
    parser = StrOutputParser()
    chain = plan_template | model | parser
    answer = chain.invoke({
        "question": question,
    })
    return (answer)

plan = create_plan("How many states in the US have capital cities that end in City?")
print(plan)

1. Create a list of all US states.
2. Create a list of all US state capitals.
3. Iterate through the list of state capitals.
4. For each capital, check if it ends with "City".
5. If it does, increment a counter.
6. Return the final value of the counter. 



In [37]:
def convert_to_code(plan): 
    step_template = PromptTemplate.from_template(
       """
       Convert the following steps into a Python function.
       {plan}
        """
    )
    parser = StrOutputParser()
    chain = step_template | model | parser
    answer = chain.invoke({"plan": plan})
    return (answer)

code = convert_to_code(plan)
print(code)

```python
def count_capitals_ending_with_city():
  """
  Counts the number of US state capitals that end with "City".

  Returns:
    int: The count of capitals ending with "City".
  """

  states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", 
            "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", 
            "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", 
            "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", 
            "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
            "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
            "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", 
            "Wisconsin", "Wyoming"]

  capitals = ["Montgomery", "Juneau", "Phoenix", "Little Rock", "Sacramento", "Denver"

In [39]:
exec('\n'.join(code.split('\n')[1:-2]))

Number of US state capitals ending with 'City': 4
